In [23]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from datetime import datetime, date, timedelta
import time
import pandas as pd
import os
from itertools import chain

In [56]:
def matches_stats_get(url):
    
    options = Options()
    #options.headless = True
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--blink-settings=imagesEnabled=false')
    #options.add_argument("--headless")

    driver = webdriver.Chrome(r'C:\Users\hp\Documents\Our Documents\Personal Development\Projects\Client Projects\Dexter Hadeveld (Upwork)\Enhancing-Sports-Insight\chromedriver-win64\chromedriver.exe',options=options)

    driver.get(url)
    #print('url: ', url)

    team_links =driver.find_elements_by_xpath("//div[@class='sc-fqkvVR gWLOaC']/div/a[@href]")
    team_link = [con.get_attribute('href') for con in team_links]
    #print('team_link: ', team_link)

    team_titles = driver.find_elements_by_class_name("sc-fqkvVR.gWLOaC")
    team_title = [con.get_attribute('innerText') for con in team_titles]
    #print('team_title: ', team_title)
    time.sleep(3)

    league_tables = driver.find_elements_by_class_name("sc-jlZhew.kgghii")
    league_table = [con.get_attribute('innerText') for con in league_tables]
    #print('league_table: ', league_table)

    driver.quit()

    # Split the data into lines and then into chunks of 5 elements each
    data = [line.split('\n') for line in league_table[0].split('\n') if line.strip()]
    data = list(chain(*data))
    #print('data: ', data)
    chunks = [data[i:i + 5] for i in range(0, len(data), 5)]

    df_league_table = pd.DataFrame(chunks, columns=['Position', 'Team', 'P', 'DIFF', 'PTS'])
    

    # Remove the first row (header row) from the DataFrame
    df_league_table = df_league_table.iloc[1:].reset_index(drop=True)


    def ind_team_stats(url):

        options = Options()
        #options.headless = True
        ua = UserAgent()
        userAgent = ua.random
        options.add_argument(f'user-agent={userAgent}')
        options.add_argument('--blink-settings=imagesEnabled=false')
        #options.add_argument("--headless")


        driver = webdriver.Chrome(r'C:\Users\hp\Documents\Our Documents\Personal Development\Projects\Client Projects\Dexter Hadeveld (Upwork)\Enhancing-Sports-Insight\chromedriver-win64\chromedriver.exe',options=options)

        driver.get(url)

        team_ages =driver.find_elements_by_class_name("sc-fqkvVR.sc-dcJsrY.jqdASS.ijIoLB")
        team_age = [con.get_attribute('innerText') for con in team_ages]
        team_age = team_age[1]
        team_age = [team_age.replace('\n', ',')]
        values = team_age[0].split(',')
        df_team_age = pd.DataFrame([values])
        df_team_age =df_team_age.drop([1, 3,5,7], axis=1)
        df_team_age.rename(columns = {0:'Total Players', 2:'Average player age',
                                      4:'Foreign players',6:'National team players'}, inplace = True)


        team_summarys =driver.find_elements_by_class_name("sc-fqkvVR.Ksgce")
        team_summary = [con.get_attribute('innerText') for con in team_summarys]
        df_team_summary = pd.DataFrame()
        # Loop through the list and split each element by '\n'
        for item in team_summary:
            values = item.split('\n')
            headers = values[::2]
            values = values[1::2]
            # Create a temporary DataFrame from the current item
            temp_df = pd.DataFrame([values], columns=headers)
            # Append the temporary DataFrame to the main DataFrame
            df_team_summary = pd.concat([df_team_summary, temp_df], axis=1)
        df_team_summary = df_team_summary.transpose()
        df_team_summary.index.name = 'Metric'
        df_team_summary.rename(columns = {0:'Values'}, inplace = True)
        df_team_summary = df_team_summary.transpose()



        previous_links =driver.find_elements_by_xpath("//div[@class='sc-fqkvVR clkedU']/div/a[@href]")
        previous_link = [con.get_attribute('href') for con in previous_links]

        previous_matches =driver.find_elements_by_xpath("//div[@class='sc-fqkvVR clkedU']/div/a")
        previous_match = [con.get_attribute('innerText') for con in previous_matches]
        match  = [item.replace('\n', ',').split(',') for item in previous_match]
        data = [sublist[:4] for sublist in match]
        previous_df = pd.DataFrame(data, columns=['Date', 'Time', 'Home Team', 'Away Team'])
        previous_df['previous_link'] = previous_link
        previous = previous_df.loc[previous_df.Time == 'FT', :].tail(1)


        driver.get(previous.previous_link.values[0])
        last_games =driver.find_elements_by_class_name("sc-fqkvVR.fVuJXd")
        last_game = [con.get_attribute('innerText') for con in last_games]


        team_names =driver.find_elements_by_class_name("sc-fqkvVR.gWLOaC")
        team_name = [con.get_attribute('innerText') for con in team_names]

        driver.quit()
        # Remove 'What is this?' from the list
        data_list = [item for item in last_game if item != 'What is this?']
        # replace \n with , and split them into individual strings
        clean = [item.replace('\n', ',').split(',') for item in data_list]
        # unnest the list
        clean = list(chain(*clean))
        #categorise them in 3's
        chunks = [clean[i:i + 3] for i in range(0, len(clean), 3)]
        last_game_df = pd.DataFrame(chunks, columns=[f'{team_name[0]}', 'Metric', f'{team_name[1]}'])

        columns_to_keep = ['Metric'] + [col for col in last_game_df.columns if any(word in col for word in team_title)]
        #print(columns_to_keep)
        last_game_df = last_game_df[columns_to_keep]
        last_game_df.set_index('Metric', inplace=True)
        last_game_df= last_game_df.transpose()
        df_team_summary.reset_index(drop=True, inplace=True)
        last_game_df.reset_index( inplace=True)
        new_last_game_df = pd.concat([last_game_df,df_team_summary], axis=1)
        new_last_game_df = new_last_game_df.rename(columns={'index': 'last_game_grounds'})


        #print(df_team_age)
        return new_last_game_df


    # Loop through the team links
    for i in range(0, len(team_link), 2):
        home_url = team_link[i]
        away_url = team_link[i + 1]

        # Fetch data for the home team
        home_df = ind_team_stats(home_url)
        home_df.reset_index(drop=True, inplace=True)
        home_df.columns = ['Home_' + col for col in home_df.columns]

        away_df = ind_team_stats(away_url)
        away_df.reset_index(drop=True, inplace=True)
        away_df.columns = ['Away_' + col for col in away_df.columns]

    # Concatenate the home and away DataFrames horizontally
    teams_df = pd.concat([home_df, away_df], axis=1)
    teams_df.columns =teams_df.columns.str.replace(' ', '_')
    return teams_df


In [84]:
def total_match_stats(leagues_list, today_date):
    leagues_dataset = {} #Created the empyt dictionary that will be used to concatenate all table from all leagues
    
    for key in list(leagues_list.keys()):
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        options.add_argument(f'user-agent={userAgent}')
        options.add_argument('--blink-settings=imagesEnabled=false')
        #options.add_argument("--headless")
        
        league_url = leagues_list[key][2]


        driver = webdriver.Chrome(r'C:\Users\hp\Documents\Our Documents\Personal Development\Projects\Client Projects\Dexter Hadeveld (Upwork)\Enhancing-Sports-Insight\chromedriver-win64\chromedriver.exe',options=options)

        driver.get(league_url)
        time.sleep(3)
        matches =driver.find_elements_by_xpath("//div[@class='sc-fqkvVR clkedU']/div/a")
        match = [con.get_attribute('innerText') for con in matches]
        match_links =driver.find_elements_by_xpath("//div[@class='sc-fqkvVR clkedU']/div/a[@href]")
        match_link = [con.get_attribute('href') for con in match_links] 

        driver.quit()
        match  = [item.replace('\n', ',').split(',') for item in match]

        data = [sublist[:4] for sublist in match]
        df = pd.DataFrame(data, columns=['Date', 'Time', 'Home Team', 'Away Team'])
        df['match_links'] = match_link
        date_list = list(df['Date'])
        date_list = [dates.split('/') for dates in date_list]
        date_list = [dates[0]+'/'+dates[1]+'/20'+dates[2] for dates in date_list]
        df['Date'] = date_list
        df['Date'] = pd.to_datetime(df['Date'],  format='%d/%m/%Y')

        #Filter rows with today's date
        today_df = df[df['Date'].dt.date == today_date] #Account for when the dataset filter everything due to no matching date
        today_df = today_df.copy(deep=True)
        curr_league = [key for i in range(len(today_df['match_links']))]
        today_df['League'] = curr_league

        combined_df = []

        for index in range(len(list(today_df.match_links))):
            url = list(today_df.match_links)[index]
            data_df = matches_get(url)
            print(list(today_df[['Date','Home Team','Away Team']].iloc[index,:]))
            data_df['Date'] = list(today_df['Date'])[index]
            data_df['Home Team'] = list(today_df['Home Team'])[index]
            data_df['Away Team'] = list(today_df['Away Team'])[index]
            data_df['League'] = list(today_df['League'])[index]

            combined_df.append(data_df)

        combined = pd.concat(combined_df, axis=0, join='outer', ignore_index=True)

        leagues_dataset[key] = combined
        
    list_of_keys = list(leagues_dataset.keys())
    if len(list_of_keys) > 0:
        for i in range(len(list_of_keys)):
            if i == 0:
                key = list_of_keys[i]
                final_dataset = leagues_dataset[key].copy(deep=True)
            else:
                key = list_of_keys[i]
                final_dataset = pd.concat([final_dataset, leagues_dataset[key]], axis=0)

        return final_dataset
    else:
        print('No league scheduled matches were extracted!')

In [80]:
testleagues_list = {
    'English Premier League':['https://www.flashscore.nl/voetbal/engeland/premier-league/schema/', 'https://www.worldfootball.net/all_matches/eng-premier-league-2023-2024/', 'https://www.sofascore.com/tournament/football/england/premier-league/17#52186'],
    #'EFL Championship':['https://www.flashscore.nl/voetbal/engeland/efl-cup/schema/', ''],
    #'EFL Trophy':['https://www.flashscore.nl/voetbal/engeland/efl-trophy/schema/', 'https://www.worldfootball.net/all_matches/eng-efl-trophy-2023-2024/'],
    #'Ligue 1':['https://www.flashscore.nl/voetbal/frankrijk/ligue-1/schema/', 'https://www.worldfootball.net/all_matches/fra-ligue-1-2023-2024/'],
    #'La Liga':['https://www.flashscore.nl/voetbal/spanje/laliga/schema/','https://www.worldfootball.net/all_matches/esp-primera-division-2023-2024/'],
    #'La Liga2':['https://www.flashscore.nl/voetbal/spanje/laliga2/schema/', 'https://www.worldfootball.net/all_matches/esp-segunda-division-2023-2024/']
    #'Serie B':['https://www.flashscore.nl/voetbal/italie/serie-b/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-b-2023-2024/'],
    #'Major League Soccer':['https://www.flashscore.nl/voetbal/usa/mls/schema/', 'https://www.worldfootball.net/all_matches/usa-major-league-soccer-2023/'],
}

In [81]:
today = date.today()
tomorrow = date.today() + timedelta(days=2)

In [82]:
print(today, tomorrow)

2023-09-14 2023-09-16


In [85]:
%%time
final_dataset = total_match_stats(testleagues_list, tomorrow)

[Timestamp('2023-09-16 00:00:00'), 'Wolves', 'Liverpool']
[Timestamp('2023-09-16 00:00:00'), 'West Ham', 'Man City']
CPU times: total: 297 ms
Wall time: 2min 8s


In [86]:
final_dataset

,Home_last_game_grounds,Home_Matches,Home_Goals_scored,Home_Goals_conceded,Home_Assists,Home_Goals_scored,Home_Penalty_goals,Home_Free_kick_goals,Home_Goals_from_inside_the_box,Home_Goals_from_outside_the_box,...,Away_throwIns,Away_goalKicks,Away_Offsides,Away_Fouls,Away_Yellow,Away_Red_cards,Date,Home Team,Away Team,League
0,Wolves Away,4,4,8,4,1,0,0/1,4/45,0/17,...,15,10.3,3,11.5,1.5,2,2023-09-16,Wolves,Liverpool,English Premier League
1,West Ham Away,4,9,4,7,2.3,1/1,0/0,8/35,1/14,...,16,5.3,1.3,8.8,0.8,0,2023-09-16,West Ham,Man City,English Premier League


In [73]:
final_dataset['League']

0    English Premier League
1    English Premier League
Name: League, dtype: object

In [87]:
#Check if the scheduled matches loaded is by date and includes the needed current date.